In [10]:
import pandas as pd 
import tika
from tika import parser
from bs4 import BeautifulSoup
from html.parser import HTMLParser


In [11]:
def get_html(doc_file):
    parsed = parser.from_file(doc_file, xmlContent=True)
    return parsed.get('content')

In [12]:
doc_file = './undp popp docs/HR_Non-Staff_Service Contract.docx'

In [13]:
html_doc = get_html(doc_file)

2022-11-07 13:53:17,516 [MainThread  ] [WARNI]  Failed to see startup log message; retrying...


In [14]:
html_doc[:200]

'<html xmlns="http://www.w3.org/1999/xhtml">\n<head>\n<meta name="date" content="2022-10-07T07:45:00Z" />\n<meta name="Total-Time" content="19" />\n<meta name="extended-properties:AppVersion" content="16.0'

In [39]:

def get_links(html):
    class URLSeeker(HTMLParser):
        def __init__(self):
            HTMLParser.__init__(self)
            self.urls = []

        def handle_starttag(self, tag, attrs):
            href = dict(attrs).get("href")
            if href and tag == "a":
                self.urls.append(href)

    url_seeker = URLSeeker()
    url_seeker.feed(html)
    return url_seeker.urls


def get_links2(html):
    soup = BeautifulSoup(html)
    results = []
    for link in soup.find_all('a'):
        meta_doc = {}
        url = link.get('href')
        if not url:
            continue
        link_str = link.string
        meta_doc = {'Link URL': url, 'Link Name': link_str}
        if 'popp.undp.org' in url:
            meta_doc.update({'Link Category': 'POPP'})
        elif 'intranet.undp.org' in url:
            meta_doc.update({'Link Category': 'Intranet'})
        elif 'content.undp.org' in url:
            meta_doc.update({'Link Category': 'UNDP System'})
        elif 'mailto:' in url:
            meta_doc.update({'Link Category': 'e-mail address'})
        else:
             meta_doc.update({'Link Category': 'Other'})

        results.append(meta_doc)
    return results

In [40]:
r = get_links2(html_doc)

In [41]:
r[0]

{'Link URL': 'https://popp.undp.org/_Layouts/15/POPPOpenDoc.aspx?ID=POPP-11-2878',
 'Link Name': 'Annex VII',
 'Link Category': 'POPP'}

In [42]:
df = pd.DataFrame(r)

In [43]:
df

,Link URL,Link Name,Link Category
0,https://popp.undp.org/_Layouts/15/POPPOpenDoc....,Annex VII,POPP
1,https://popp.undp.org/_Layouts/15/POPPOpenDoc....,Annex I,POPP
2,https://popp.undp.org/_Layouts/15/POPPOpenDoc....,Annex I,POPP
3,http://content.undp.org/go/userguide/results/,http://content.undp.org/go/userguide/results/,UNDP System
4,https://popp.undp.org/_Layouts/15/POPPOpenDoc....,Annex IV,POPP
5,https://popp.undp.org/_Layouts/15/POPPOpenDoc....,Annex VI,POPP
6,https://popp.undp.org/_Layouts/15/POPPOpenDoc....,Annex III,POPP
7,https://intranet.undp.org/unit/ofrm/_layouts/1...,proforma costs for SC holders.,Intranet
8,https://popp.undp.org/_Layouts/15/POPPOpenDoc....,Handbook on Setting Remuneration for Service C...,POPP
9,https://intranet.undp.org/unit/ohr/learning/Si...,https://intranet.undp.org/unit/ohr/learning/Si...,Intranet


In [44]:
df['Link Category'].value_counts().to_frame()

,Link Category
POPP,24
Intranet,2
e-mail address,2
Other,2
UNDP System,1
